In [1]:
import pickle
import numpy as np
import pandas as pd

### First, merge the two files that you have

Note to **Farnoud**: please skip to later part, there is a subset sample, so you can load easily

In [2]:
with open(r"C:\Users\zmich\OneDrive - Uniwersytet Jagielloński\Szkoła Doktorska\Miscellaneous\final_res_16-09-22.obj", "rb") as file:
    data1 = pickle.load(file)

In [3]:
with open(r"C:\Users\zmich\OneDrive - Uniwersytet Jagielloński\Szkoła Doktorska\Miscellaneous\final_res_03-11-22.obj", "rb") as file:
    data2 = pickle.load(file)

In [4]:
data = data1 + data2

In [5]:
data.sort(key=lambda x: x["settings"]["Batch"] * 5 + x["settings"]["shared_discount"])

In [6]:
data[3]["settings"]

{'Replication': 0,
 'Batch': 0,
 'shared_discount': 0.32,
 'Start_time': Timestamp('2016-01-01 00:00:17'),
 'End_time': Timestamp('2016-01-01 00:29:45'),
 'Demand_size': 198}

optionally save the object

In [39]:
with open(r"some_path\some_name.obj", "wb") as file:
    pickle.dump(data, file)

## Adding long and lat

**data** is the list with all results

To add longitudes and latitudes, we need to load one full dotmap with exmas results

In [7]:
with open(r"C:\Users\zmich\OneDrive - Uniwersytet Jagielloński\Szkoła Doktorska\Miscellaneous\single_full_dotmap_29-11-22.obj", "rb") as file:
    one_dataset = pickle.load(file)[0]

Small sample for the purpose of recreation: **Farnoud starts here** <br>
For **full analysis ommit the lines**

In [2]:
with open("data_storage/one_dataset_cut.obj", "rb") as file:
    one_dataset = pickle.load(file)

In [3]:
with open("data_storage/small_dataset.obj", "rb") as file:
    data = pickle.load(file)

**Start back here**

In [4]:
nodes_geo = pd.DataFrame({"Node": [t[0] for t in one_dataset["G"].nodes._nodes.items()],
                          "x": [t[1]["x"] for t in one_dataset["G"].nodes._nodes.items()],
                          "y": [t[1]["y"] for t in one_dataset["G"].nodes._nodes.items()]})
nodes_geo[:3]

,Node,x,y
0,42421728,-73.960044,40.798048
1,42421731,-73.961474,40.798654
2,42421737,-73.962873,40.799244


In [5]:
def merging_foo(data, nodes_geo):
    new_requests = data["requests"].merge(nodes_geo, left_on="origin", right_on="Node")\
                    .drop(columns="Node").rename(columns={"x": "x_org", "y": "y_org"})
    new_requests = new_requests.merge(nodes_geo, left_on="destination", right_on="Node")\
                    .drop(columns="Node").rename(columns={"x": "x_dest", "y": "y_dest"})
    data["requests"] = new_requests

In [6]:
for dt in data:
    merging_foo(dt, nodes_geo)

In [7]:
data[0]["requests"][:3]

,id,pickup_datetime,dropoff_datetime,trip_duration,origin,destination,status,dist,treq,ttrav,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,id0190469,2016-01-01 00:00:17,2016-01-01 00:14:26,849,42427352,42438018,0,14340,0,2390,...,999999,306,2591,29.772825,21,0,-73.981942,40.719462,-73.938959,40.828984
1,id1210365,2016-01-01 00:01:01,2016-01-01 00:07:49,408,42432756,42434113,0,2640,44,440,...,999999,1,440,5.500000,1,0,-73.965196,40.801107,-73.947489,40.815207
2,id3991965,2016-01-01 00:20:08,2016-01-01 00:29:16,548,42437917,42434113,0,2705,1191,450,...,999999,118,450,5.632500,1,0,-73.963370,40.798568,-73.947489,40.815207


optional:

In [23]:
with open("some_path\some_name2.obj", "wb") as file:
    pickle.dump(data, file)

Either loop similarly to earlier or create a great df as follows

In [8]:
concatenated_data = pd.concat([d["requests"] for d in data])

In [9]:
concatenated_data[:3]

,id,pickup_datetime,dropoff_datetime,trip_duration,origin,destination,status,dist,treq,ttrav,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,id0190469,2016-01-01 00:00:17,2016-01-01 00:14:26,849,42427352,42438018,0,14340,0,2390,...,999999,306,2591,29.772825,21,0,-73.981942,40.719462,-73.938959,40.828984
1,id1210365,2016-01-01 00:01:01,2016-01-01 00:07:49,408,42432756,42434113,0,2640,44,440,...,999999,1,440,5.500000,1,0,-73.965196,40.801107,-73.947489,40.815207
2,id3991965,2016-01-01 00:20:08,2016-01-01 00:29:16,548,42437917,42434113,0,2705,1191,450,...,999999,118,450,5.632500,1,0,-73.963370,40.798568,-73.947489,40.815207


In [26]:
len(concatenated_data)

4908349